# Unconstrained Optimization

This notebook with explore different algorithms for unconstrained optimization, specifically those which gradient (1st derivative) and second derivative information are available. It will open with a simple introduction of line search which extends to various related algorithms and then trust region methods. Additional algorithms maybe appended afterwards.

To compute 1st and 2nd order derivatives, **autograd** package will be used.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from autograd import grad
from scipy.optimize import minimize #for solving trust region subproblem

## Line Search

### Naive Intuition

Line search are a type of algorithms where given its initial point, it will seek "along a line" to find the local minima of a given objective function. 

There are a few things to consider when using line search algorithms:

1. Initialization point
2. Step size
3. Direction
4. Terminal condition (when to stop)

Let's create a naive line search algorithm to get the basic intuition with the preceeding items address as the following:

1. Initialization point - algo input
2. Step size - algo inpu
3. Direction - computed via gradient of the objective function
4. Terminal condition - algo input as number of iterations

In [2]:
def naive_line_search(objective_function, initial_point, step_size, iterations):
    '''
    returns arguments which identifies local minima of given objective function given 
        initial points, 
        step size, 
        and iterations
    '''
    
    x = np.array([float(i) for i in initial_point])
    i = 0
    
    while i < iterations:
        
        gradient = np.array(grad(objective_function)(x))*-1 # descent direction determined by negative gradient
        unit_direction = gradient/np.linalg.norm(gradient) # desecnt direction factor effect minized to unit vector
        x = list(x + unit_direction*step_size) # x is updated with x + step_size*direction
        
        i += 1
        
    return x

With the above naive function, lets try the algorithm on f(x,y) = x^2 + y^2 where the minimum is at (0,0).

In [3]:
def obj_fn(x):
    return x[0]**2 + x[1]**2

naive_line_search(obj_fn, [10, 5], 0.01, 2000)

[0.0003040046208548086, 0.0001520023104274043]

We see that this is subsequently really close, lets see what happens if we were to change some parameters.

In [4]:
naive_line_search(obj_fn, [37, 50], 0.01, 2000)

[25.103139891794168, 33.92316201594099]

In [5]:
naive_line_search(obj_fn, [37, 50], 1, 2000)

[0.11973366456175838, 0.16180224940778176]

In [6]:
naive_line_search(obj_fn, [37, 50], 2, 2000)

[-1.0699523462588483, -1.4458815489984427]

In [7]:
naive_line_search(obj_fn, [37, 50], 1, 100)

[0.11973366456175838, 0.16180224940778176]

In [8]:
naive_line_search(obj_fn, [37, 50], 2, 100)

[-1.0699523462588483, -1.4458815489984427]

In [9]:
naive_line_search(obj_fn, [370, 400], 0.01, 2000)

[356.4191603676318, 385.3180112082782]

So we notice here that the results greatly depend on parameters fed into the algorithm. Which sort of implies, that this is a guessing game if the nature of the objective function is not already known. Is there some way we can optimize the algorithm to limit the amount of parameters needed?

Let's try setting a terminal/stopping condition instead of a manual input of iterations.

In [10]:
def self_terminating_line_search(objective_function, initial_point, step_size):
    '''
    returns arguments which identifies local minima of given objective function given 
        initial points, 
        step size
    '''
    
    x0 = np.array([float(i) for i in initial_point])
    stop = False
    
    while stop == False:
        
        gradient = np.array(grad(objective_function)(x0))*-1 # descent direction determined by negative gradient
        unit_direction = gradient/np.linalg.norm(gradient) # desecnt direction factor effect minized to unit vector
        x = list(x0 + unit_direction*step_size) # x is updated with x + step_size*direction
        
        if objective_function(x0) <= objective_function(x):
            stop = True #stop "searching" if the results of the next input values is bigger or equal to the previous input value
        else:
            x0 = x
        
    return x

In [11]:
%%time
self_terminating_line_search(obj_fn, [37, 50], 0.01)

CPU times: user 1.64 s, sys: 29.3 ms, total: 1.67 s
Wall time: 1.62 s


[-0.005183366570926999, -0.007004549420172505]

In [12]:
%%time
naive_line_search(obj_fn, [37, 50], 0.01, 2000)

CPU times: user 609 ms, sys: 54 ms, total: 663 ms
Wall time: 587 ms


[25.103139891794168, 33.92316201594099]

We see that although we get a better result and have 1 less parameter we need to "guess", if we start off with not so good parameters, the algorithm can take forever to run. Case and point if I were to change the step size.

In [13]:
%%time
self_terminating_line_search(obj_fn, [37, 50], 0.05)

CPU times: user 435 ms, sys: 879 µs, total: 436 ms
Wall time: 379 ms


[-0.028977086790774598, -0.03915822539293959]

So in this case, is there a way we can improve (perhaps optimize) our selection of step size?

### Armijo + Wolfe Conditions

The Armijo and subsequently Wolfe conditions allows for the descent algorithm to select a better suited step length for every iteration (after the initialization) with given parameters (Armijo: c1, Wolfe: c1, c2, all 0 < c < 1 for all c). 

Below is the Armijo condition function:

In [14]:
def armijo(objective_function, x_current, search_direction, c1, initial_stepsize):
    
    stepsize = initial_stepsize
    x_next = x_current + stepsize*search_direction
    f_current = objective_function(x_current)
    f_next = objective_function(x_next)
    gradient_current = np.array(grad(objective_function)(x_current))
    
    while ((f_next) > (f_current + c1 * stepsize * np.dot(gradient_current, search_direction))):
        stepsize = stepsize*0.5
        x_next = x_current + stepsize * search_direction
        f_next = objective_function(x_next)
        
    return stepsize

As you can see, it is meant to take each input iteration, and return the ideal step_size to continue next descent iteration.

Below is the Wolfe condition function:

In [15]:
def wolfe(objective_function, x_current, search_direction, c1, c2, initial_stepsize):
    
    stepsize = initial_stepsize
    x_next = x_current + stepsize*search_direction
    f_current = objective_function(x_current)
    f_next = objective_function(x_next)
    gradient_current = np.array(grad(objective_function)(x_current))
    gradient_next = np.array(grad(objective_function)(x_next))
    
    lower_bound = 0
    upper_bound = np.inf
    
    while True:
        if ((f_next) > (f_current + c1 * stepsize * np.dot(gradient_current, search_direction))):
            upper_bound = stepsize
            stepsize = (lower_bound + upper_bound)*0.5
        elif (np.dot(gradient_next, search_direction) < c2 * np.dot(gradient_current, search_direction)):
            lower_bound = stepsize
            if np.isinf(upper_bound):
                stepsize = lower_bound*2.0
            else:
                stepsize = (lower_bound + upper_bound)*0.5
        else:
            break
        
        x_next = x_current + stepsize * search_direction
        f_next = objective_function(x_next)
        gradient_next = np.array(grad(objective_function)(x_next))
    
    return stepsize

Let's incorporate these into descent algorithm.

In [16]:
def aw_line_search(objective_function, initial_point, c1, c2, initial_stepsize, condition='armijo'):
    x0 = np.array([float(i) for i in initial_point])
    stop = False
    
    while stop == False:
        gradient = np.array(grad(objective_function)(x0))*-1 # descent direction determined by negative gradient
        unit_direction = gradient/np.linalg.norm(gradient) # desecnt direction factor effect minized to unit vector
        
        # find stepsize to calculate next x
        if condition.lower() == 'armijo':
            stepsize = armijo(objective_function, x0, unit_direction, c1, initial_stepsize)
        elif condition.lower() == 'wolfe':
            stepsize = wolfe(objective_function, x0, unit_direction, c1, c2, initial_stepsize)
        else:
            print('condition name required')
            break
        
        x = list(x0 + stepsize * unit_direction)
        
        if objective_function(x0) <= objective_function(x):
            stop = True #stop "searching" if the results of the next input values is bigger or equal to the previous input value
        else:
            x0 = x
    
    return x

In [17]:
%%time
aw_line_search(obj_fn, [37, 50], 0.1, 0.75, 0.01, condition='armijo')

CPU times: user 3.66 s, sys: 57.7 ms, total: 3.72 s
Wall time: 3.66 s


[5.346248810556834e-163, 7.224660554807402e-163]

In [18]:
%%time
aw_line_search(obj_fn, [37, 50], 0.1, 0.75, 0.01, condition='wolfe')

CPU times: user 13.4 s, sys: 11.5 ms, total: 13.4 s
Wall time: 13.4 s


[8.060513460866673e-163, 1.0892585757926226e-162]

As we see here, both the use of Armijo and Wolfe conditions yielded superior results comparing to the naive cases earlier.
However, it was done so with the expense of more time (another loop to search for ideal stepsize for every descent iterations) and it doesn't really relieve the issues of guessing parameters (in this case hyperparameters c1 and c2, depending on condition). So there is some sort of trade off here, if precision is much more important, then the usage of the conditions are recommended, if not then naive approach maybe a better idea. This is potentially why hyperparameter tuning of machine learning model training are on the more naive end of the spectrum in practice.

Furthermore, if the wrong hyperparameters (c1 and c2) are used, the descent path can deteriorate leading to an invalid solution.

In [19]:
%%time
aw_line_search(obj_fn, [37, 50], 0.01, 0.75, 0.01, condition='armijo')

CPU times: user 3.85 s, sys: 39.7 ms, total: 3.89 s
Wall time: 3.87 s


<ipython-input-16-adea20f55111>:7: RuntimeWarning: divide by zero encountered in true_divide
  unit_direction = gradient/np.linalg.norm(gradient) # desecnt direction factor effect minized to unit vector
<ipython-input-14-05f488395449>:9: RuntimeWarning: invalid value encountered in double_scalars
  while ((f_next) > (f_current + c1 * stepsize * np.dot(gradient_current, search_direction))):


[-inf, -inf]

In [20]:
%%time
aw_line_search(obj_fn, [37, 50], 0.01, 0.75, 0.01, condition='wolfe')

<ipython-input-16-adea20f55111>:7: RuntimeWarning: divide by zero encountered in true_divide
  unit_direction = gradient/np.linalg.norm(gradient) # desecnt direction factor effect minized to unit vector


CPU times: user 16.8 s, sys: 0 ns, total: 16.8 s
Wall time: 16.7 s


<ipython-input-15-d52f5df1e5bd>:14: RuntimeWarning: invalid value encountered in double_scalars
  if ((f_next) > (f_current + c1 * stepsize * np.dot(gradient_current, search_direction))):


[inf, inf]

### Newton Method

If the 2nd derivative (or hessian) can be derived from the objective function, then the descent direction can be computed using both the hessian and gradient. The idea here is to use Newton method of root finding to identify roots of the gradient (i.e. set of inputs which have gradient = 0).

In [21]:
from autograd import hessian
from numpy.linalg import inv

def newton_method(objective_function, initial_point, stepsize, iterations):
    i = 0
    x = np.array([float(i) for i in initial_point])
    
    while i < iterations:
        gradient = np.array(grad(objective_function)(x)) 
        h = hessian(objective_function)(np.array(x))
        
        direction = -1.0*np.dot(inv(h),gradient) 
        
        x = list(x + direction*stepsize) # x is updated with x + step_size*direction
        i += 1
    
    return x

In [22]:
%%time
newton_method(obj_fn, [37, 50], 1, 2000) 

CPU times: user 2.66 s, sys: 0 ns, total: 2.66 s
Wall time: 2.65 s


[0.0, 0.0]

In [23]:
%%time
newton_method(obj_fn, [37, 50], 0.1, 2000) 

CPU times: user 2.64 s, sys: 18 ms, total: 2.65 s
Wall time: 2.63 s


[1.1303699476614377e-90, 1.5275269562992385e-90]

In [24]:
%%time
newton_method(obj_fn, [37, 50], 0.01, 2000) 

CPU times: user 2.22 s, sys: 40.2 ms, total: 2.26 s
Wall time: 2.25 s


[6.895899431071434e-08, 9.318783014961302e-08]

In the event where 2nd order derivative is not computable, a "secant" method can be used where the 2nd order derivatives are approximated by the secants of 1st order derivates. I will not dive down this rabbit hole for the time being.

## Trust Region Methods

What if a skeptic comes along and say algorithms shouldn't place too much trust on first-order and second-order conditions? What if a large magnitude of gradient results in overstepping of the descent? (i.e. skipping over the local minima). Trust region methods are employed where the descent iterations are more restricted (preventing overstepping). It predicts both the movemnt associated with the step and also limits the step taken.

Trust region methods choose to determine step size prior to determining step direction unlike line search methods. The name trust region comes from determining the next step by minimizing the objective function around an area of the current iteration. 

Trust region methods also examines on whether the resulting prediction increment is "good enough" to determine whether there should be an increase in radius to consider for next increment estimation.

Below is a sample of a trust region descent method.

In [25]:
def solve_trust_region_subproblem(objective_function, gradient_x, hessian_x, x, delta, lagrange_multiplier_init = 1):
    '''
    minimizes function m(p) = f(k) + f'(k)*p + 1/2*f''(k)*p^2
    such that norm(p) <= delta
    
    p will be the step length of the next step
    '''
    
    # p_prime = solved optimization problem
    
    x_prime = x + p_prime
    y_prime = obj_fn(x) + np.dot(gradient_x, p_prime) + 0.5*np.matmul(np.matmul(np.transpose(p_prime), hessian_x), p_prime) 
    
    return x_prime, y_prime

def trust_region_descent(objective_function, x, k_max, eta_1, eta_2, gamma_1, gamma_2, delta):
    '''
    eta_1 = threshhold and how much to shrink radius with bad prediction
    eta_2 = threshhold and how much to increase  radius with good prediction
    delta = initial region radius
    '''
    
    y = objective_function(x)
    g = np.array(grad(objective_function)(x))
    h = np.array(hessian(objective_function)(x))
    
    for k in range(0, k_max):
        x_prime, y_prime = solve_trust_region_subproblem(objective_function, g, h, x, delta)
        
        print(x_prime, y_prime)
        trust_region_radius_ratio = (y - objective_function(x_prime))/(y - y_prime)
        
        if trust_region_radius_ratio < eta_1:
            delta = gamma_1*delta # shrinks next radius of region if actual drop is worse than expected
        else:
            x = x_prime # updates x if actual drop is better than expected
            
            y = objective_function(x)
            g = np.array(grad(objective_function)(x))
            h = np.array(hessian(objective_function)(x))
            
            if trust_region_radius_ratio > eta_2:
                delta = gamma_2*delta # increase next radius of region if actual drop is better than expected
    
    return x


Note that the subproblem in trust region is a constrained optimization problem with inequality constraints. Therefore, the code is only used to illustrate the intuition of the algorithm.